<a href="https://colab.research.google.com/github/kanade2001/KokomeloTalk/blob/g0roro%2Festimate_emotion/backend/public_emotion_estimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 必要なモジュールのインストール/インポート
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
! pip install fugashi ipadic
! pip install unidic_lite
import numpy as np
import os
import gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 21.4 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=fb8ec0da78ba49ce11c880b552d67eee920db29d9a296817424ff26bdc870e2e
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=7e038e4e4b5cdd10392ba798f3b568ec48f202a952d7d44dbffd48784ddaf562
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic_lite


In [2]:
# フォルダIDを指定
folder_id = "1YmF_3TihtXOEELXUr_ShG1kT4rIIzgef"

# ダウンロード先ディレクトリ
save_dir = "/content/trained_model"
os.makedirs(save_dir, exist_ok=True)

# gdownを使ってフォルダ全体をダウンロード
gdown.download_folder(f"https://drive.google.com/drive/folders/{folder_id}", output=save_dir, quiet=False)


Retrieving folder contents


Processing file 1-2LeATWCSnzrmj94qXuXNy2Kkg1dWYRM config.json
Processing file 1-4v4SmOZpP9qptoIRGtA9HxviXk3loG3 model.safetensors
Processing file 1-6ZRfqGyU5tvVfMLVz09lY6zACI0TZsX special_tokens_map.json
Processing file 1-FXYSX3WHD7odmWW7kgSJS0q4XVCA0Dc tokenizer_config.json
Processing file 1-604zi6s3vEQ7bOM8B8tAbKhXJWpDP0A vocab.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-2LeATWCSnzrmj94qXuXNy2Kkg1dWYRM
To: /content/trained_model/config.json
100%|██████████| 1.11k/1.11k [00:00<00:00, 2.99MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-4v4SmOZpP9qptoIRGtA9HxviXk3loG3
From (redirected): https://drive.google.com/uc?id=1-4v4SmOZpP9qptoIRGtA9HxviXk3loG3&confirm=t&uuid=3dfb03df-86cb-4f86-98c8-3595305b614f
To: /content/trained_model/model.safetensors
100%|██████████| 443M/443M [00:03<00:00, 113MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-6ZRfqGyU5tvVfMLVz09lY6zACI0TZsX
To: /content/trained_model/special_tokens_map.json
100%|██████████| 125/125 [00:00<00:00, 327kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-FXYSX3WHD7odmWW7kgSJS0q4XVCA0Dc
To: /content/trained_model/tokenizer_config.json
100%|██████████| 1.37k/1.37k [00:00<00:00, 3.95MB/s]
Downloading...
F

['/content/trained_model/config.json',
 '/content/trained_model/model.safetensors',
 '/content/trained_model/special_tokens_map.json',
 '/content/trained_model/tokenizer_config.json',
 '/content/trained_model/vocab.txt']

In [4]:
# モデルとトークナイザを読み込み
from transformers import AutoModelForSequenceClassification, AutoTokenizer
loaded_model = AutoModelForSequenceClassification.from_pretrained(save_dir)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_dir)

# デバイスを指定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
# 感情名（日本語）を定義
emotion_names_jp = ['喜び', '悲しみ', '怒り', '恐れ', '嫌悪', '驚き', '信頼', '期待'] # 例として8つの感情を想定

# ソフトマックス関数
def np_softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

def analyze_emotion(text):
    # 推論モードを有効化
    loaded_model.eval()

    # 入力データ変換 + 推論
    tokens = loaded_tokenizer(text, truncation=True, return_tensors="pt")
    tokens.to(loaded_model.device)
    preds = loaded_model(**tokens)
    prob = np_softmax(preds.logits.cpu().detach().numpy()[0])
    out_dict = {n: p for n, p in zip(emotion_names_jp, prob)}
    out_dict_ud = sorted(out_dict.items(), key=lambda x:x[1], reverse=True)

    print("入力文：", text)
    print(out_dict_ud)

analyze_emotion('昨日彼女に振られました')

入力文： 昨日彼女に振られました
[('悲しみ', 0.9329988), ('信頼', 0.039915565), ('驚き', 0.01132242), ('恐れ', 0.00851324), ('喜び', 0.003976579), ('嫌悪', 0.002052537), ('怒り', 0.0010010423), ('期待', 0.0002198929)]
